## Implémenter un modèle GPT from scratch pour générer du texte


Nous avons déjà étudié et codé le mécanisme d'attention multi-tête, l'un des composants centraux des LLM. Nous allons maintenant coder les autres briques d'un LLM et les assembler dans un modèle de type GPT que nous entraînerons au prochain chapitre pour générer du texte proche de celui d'un humain.


In [ ]:
import torch
import torch.nn as nn
from gptlight.tokenizer import GPTTokenizer

torch.set_printoptions(sci_mode=False)

### Coder une architecture de LLM


Nous définissons la configuration du petit modèle GPT-2 via le dictionnaire Python suivant, que nous réutiliserons plus loin dans les exemples de code :


In [19]:
GPT_CONFIG_124M = {
    "vocab_size": 50257, # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768, # Embedding dimension
    "n_heads": 12, # Number of attention heads
    "n_layers": 12, # Number of layers
    "drop_rate": 0.1, # Dropout rate
    "qkv_bias": False # Query-Key-Value bias
}

- `vocab_size` correspond à un vocabulaire de 50 257 mots, comme celui utilisé par le tokenizer BPE.
- `context_length` désigne le nombre maximal de tokens que le modèle peut traiter grâce aux embeddings positionnels.
- `emb_dim` représente la taille des embeddings, en transformant chaque token en un vecteur de dimension 768.
- `n_heads` indique le nombre de têtes d'attention dans le mécanisme multi-tête.
- `n_layers` précise le nombre de blocs transformeur dans le modèle, que nous détaillerons ensuite.
- `drop_rate` exprime l'intensité du dropout (0,1 signifie qu'environ 10 % des unités cachées sont mises à zéro de façon aléatoire) afin de limiter le surapprentissage.
- `qkv_bias` détermine s'il faut inclure un biais dans les couches linéaires du multi-head attention pour les calculs de requêtes, clés et valeurs. Nous partons par défaut sans biais, conformément aux usages des LLM modernes.


Nous commençons par l'ossature GPT, une architecture de départ, avant d'étudier chaque brique centrale et de les assembler en bloc transformeur pour obtenir l'architecture finale.


In [ ]:
class DummyGPTModel(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_block = nn.Sequential(
            *[
                DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])
            ]
        )
        self.final_norm = DummyLayernorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=cfg["qkv_bias"]
        )
    
    def forward(self, in_idx):
        
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_block(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        
        return logits
        
        
        


class DummyTransformerBlock(nn.Module):
    
    def __init__(self, cfg:dict):
        super().__init__()
    
    def forward(self, x):
        return x

class DummyLayernorm(nn.Module):
    
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
    
    def forward(self, x):
        return x

In [ ]:
tokenizer = GPTTokenizer()
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

batch = torch.stack(batch, dim=0)
batch

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

In [22]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print(f"Output shape : {logits.shape}")
print(logits)

Output shape : torch.Size([2, 4, 50257])
tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.4476,  0.1726,  0.5354,  ..., -0.3932,  1.5285,  0.8557],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.8422,  0.8243, -0.1098,  ..., -0.1434,  0.2079,  1.2046],
         [ 0.1355,  1.1858, -0.1453,  ...,  0.0869, -0.1590,  0.1552],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>)


Le tenseur de sortie comporte deux lignes correspondant aux deux exemples de texte. Chaque exemple contient quatre tokens ; chaque token est encodé comme un vecteur de dimension 50 257, ce qui correspond à la taille du vocabulaire du tokenizer.

L'embedding possède 50 257 dimensions car chacune représente un token unique du vocabulaire. Lorsque nous implémenterons le post-traitement, nous convertirons ces vecteurs à 50 257 dimensions en identifiants de tokens, que nous pourrons ensuite décoder en mots.


### Normaliser les activations avec une layer normalization


L'entraînement de réseaux profonds comportant de nombreuses couches peut se révéler difficile à cause de phénomènes tels que l'explosion ou la disparition du gradient. Ces problèmes rendent l'entraînement instable et compliquent l'ajustement efficace des poids, ce qui empêche le réseau de trouver un jeu de paramètres minimisant la fonction de perte.

Implémentons maintenant la layer normalization pour améliorer la stabilité et l'efficacité de l'entraînement. L'idée principale consiste à ajuster les activations (les sorties) d'une couche de réseau afin qu'elles aient une moyenne nulle et une variance unitaire.


> Exemple


In [28]:
torch.manual_seed(123)
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(f"out: {out}")
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
out_norm = (out-mean)/torch.sqrt(var)
print(f"\nout_norm :{out_norm}")

out: tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)

out_norm :tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)


> Une classe de layer normalization


In [30]:
class LayerNorm(nn.Module):
    
    def __init__(self, emb_dim:int):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x:torch.Tensor):
        
        x_mean = x.mean(dim=-1, keepdim=True)
        x_var = x.var(dim=-1, keepdim=True, unbiased=False)
        x_norm = (x-x_mean)/torch.sqrt(x_var + self.eps)
        
        return self.scale*x_norm + self.shift

Les paramètres de mise à l'échelle (scale) et de décalage (shift) sont entraînables (de la même dimension que l'entrée) et le LLM les ajuste automatiquement pendant l'entraînement si cela améliore ses performances.

Ainsi, le modèle peut apprendre la mise à l'échelle et le décalage les plus adaptés aux données qu'il traite.


In [32]:
layer_norm = LayerNorm(emb_dim=5)
out_norm = layer_norm(batch_example)
print(f"out_norm : {out_norm}")

mean = out_norm.mean(dim=-1, keepdim=True)
print(f"\nmean : {mean}")

var = out_norm.var(dim=-1, unbiased=False, keepdim=True)
print(f"\nvar : {var}")

out_norm : tensor([[ 0.5528,  1.0693, -0.0223,  0.2656, -1.8654],
        [ 0.9087, -1.3767, -0.9564,  1.1304,  0.2940]], grad_fn=<AddBackward0>)

mean : tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)

var : tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


### Implémenter un réseau feed-forward avec des activations GELU


Historiquement, la fonction d'activation ReLU a été largement utilisée en apprentissage profond pour sa simplicité et son efficacité. Cependant, dans les LLM, d'autres activations sont également employées, notamment GELU (Gaussian Error Linear Unit) et SwiGLU (Swish-Gated Linear Unit).


La fonction GELU peut être implémentée de plusieurs façons ; la version exacte se définit par GELU(x) = x·Φ(x), où Φ(x) est la fonction de répartition d'une loi normale standard. En pratique, on utilise souvent une approximation moins coûteuse à calculer :

$$
\mathrm{GELU}(x) \approx 0.5\,x \left( 1 + \tanh\left[ \sqrt{\frac{2}{\pi}} \left( x + 0.044715\,x^3 \right) \right] \right)
$$


> Implémentation de la fonction d'activation GELU


In [33]:
class GELU(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, x:torch.Tensor):
        
        return 0.5*x*(
            1 + torch.tanh(
                torch.sqrt(torch.tensor(2/torch.pi))*(
                    x + 0.044715*torch.pow(x, 3)
                )
            )
        )

Utilisons maintenant GELU pour implémenter le petit module FeedForward que nous intégrerons plus tard dans le bloc transformeur du LLM.


> Un module de réseau feed-forward


In [34]:
class FeedForward(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4*cfg["emb_dim"]),
            GELU(),
            nn.Linear(4*cfg["emb_dim"], cfg["emb_dim"]),
        )
    
    def forward(self, x:torch.Tensor):
        return self.layers(x)

In [35]:
ffn = FeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768)
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])


### Ajouter des connexions de raccourci


À l'origine, les connexions de raccourci ont été introduites dans les réseaux profonds de vision (notamment les réseaux résiduels) pour atténuer le problème de disparition du gradient. Ce problème apparaît lorsque les gradients, qui guident les mises à jour de poids, deviennent de plus en plus faibles en remontant les couches, ce qui rend difficile l'entraînement des premières couches.


Les connexions de raccourci consistent à ajouter l'entrée d'une couche à sa sortie, créant ainsi un chemin alternatif qui contourne certaines couches.


> Réseau illustrant les connexions de raccourci


In [44]:
class ExampleDeepNeuralNetwork(nn.Module):
    
    def __init__(self, layer_sizes, use_shortcut:bool):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList(
            [
                nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())   
            ]
        )
        
    def forward(self, x:torch.Tensor):
        for layer in self.layers:
            layer_output = layer(x)
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x

In [ ]:
layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
torch.manual_seed(123)
model_without_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=False
)

In [37]:
def print_gradients(model, x):
    output = model(x)
    target = torch.tensor([[0.]])
    
    loss = nn.MSELoss()
    loss = loss(output, target)
    
    loss.backward()
    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")


In [43]:
print_gradients(model_without_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.00020173584925942123
layers.1.0.weight has gradient mean of 0.00012011159560643137
layers.2.0.weight has gradient mean of 0.0007152040489017963
layers.3.0.weight has gradient mean of 0.0013988736318424344
layers.4.0.weight has gradient mean of 0.005049645435065031


In [45]:
torch.manual_seed(123)
model_with_shortcut = ExampleDeepNeuralNetwork(
layer_sizes, use_shortcut=True
)
print_gradients(model_with_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.22169791162014008
layers.1.0.weight has gradient mean of 0.20694105327129364
layers.2.0.weight has gradient mean of 0.32896995544433594
layers.3.0.weight has gradient mean of 0.2665732204914093
layers.4.0.weight has gradient mean of 1.3258540630340576


En résumé, les connexions de raccourci sont essentielles pour dépasser les limitations liées à la disparition du gradient dans les réseaux profonds. Elles constituent une brique incontournable des grands modèles comme les LLM.


### Relier attention et couches linéaires dans un bloc transformeur


Ce bloc, répété une douzaine de fois dans l'architecture GPT-2 à 124 millions de paramètres, combine plusieurs notions vues précédemment : attention multi-tête, layer normalization, dropout, couches feed-forward et activations GELU.


> Le bloc transformeur composant de GPT


In [2]:
from gptlight.models.transformer.normalization import LayerNorm
from gptlight.models.transformer.attention import MultiHeadAttention
from gptlight.models.transformer.ffn import FeedForward

from gptlight.config import GPTConfig, GPT2_CONFIG_124M

class GPTTransformerBlock(nn.Module):
    
    def __init__(self, cfg:GPTConfig):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg.emb_dim,
            d_out=cfg.emb_dim,
            num_heads=cfg.n_heads,
            context_length=cfg.context_length,
            dropout=cfg.drop_rate,
            qkv_bias=cfg.qkv_bias
        )
        
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(emb_dim=cfg.emb_dim)
        self.norm2 = LayerNorm(emb_dim=cfg.emb_dim)
        self.drop_shortcut = nn.Dropout(cfg.drop_rate)
    
    def forward(self, x:torch.Tensor):
        
        shortcut =  x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x) 
        x = x + shortcut
        
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        
        return x     

In [3]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768)
block = GPTTransformerBlock(GPT2_CONFIG_124M)
output = block(x)
print("Input shape: ", x.shape)
print("Output shape: ", output.shape)

Input shape:  torch.Size([2, 4, 768])
Output shape:  torch.Size([2, 4, 768])


### Coder le modèle GPT


In [ ]:
from gptlight.config import GPTConfig, GPT2_CONFIG_124M
from gptlight.models.transformer.normalization import LayerNorm
from gptlight.models.transformer import GPTTransformerBlock

class GPTModel(nn.Module):
    
    def __init__(self, cfg:GPTConfig):
        super().__init__()
        
        self.tok_emb = nn.Embedding(cfg.vocab_size, cfg.emb_dim)
        self.pos_emb = nn.Embedding(cfg.context_length, cfg.emb_dim)
        self.drop_emb = nn.Dropout(cfg.drop_rate)
        self.trf_blocks = nn.Sequential(
            *[
                GPTTransformerBlock(cfg) for _ in range(cfg.n_layers)
            ]
        )
        
        self.final_norm = LayerNorm(cfg.emb_dim)
        self.out_head = nn.Linear(cfg.emb_dim, cfg.vocab_size, bias=False)
    
    def forward(self, in_idx):
        
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        
        return logits
        
        
        

In [7]:
torch.manual_seed(123)
model = GPTModel(GPT2_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.1381,  0.0077, -0.1963,  ..., -0.0222, -0.1060,  0.1717],
         [ 0.3865, -0.8408, -0.6564,  ..., -0.5163,  0.2369, -0.3357],
         [ 0.6989, -0.1829, -0.1631,  ...,  0.1472, -0.6504, -0.0056],
         [-0.4290,  0.1669, -0.1258,  ...,  1.1579,  0.5303, -0.5549]],

        [[ 0.1094, -0.2894, -0.1467,  ..., -0.0557,  0.2911, -0.2824],
         [ 0.0882, -0.3552, -0.3527,  ...,  1.2930,  0.0053,  0.1898],
         [ 0.6091,  0.4702, -0.4094,  ...,  0.7688,  0.3787, -0.1974],
         [-0.0612, -0.0737,  0.4751,  ...,  1.2463, -0.3834,  0.0609]]],
       grad_fn=<UnsafeViewBackward0>)


À l'aide de la méthode `numel()` (abréviation de « number of elements »), nous pouvons compter le nombre total de paramètres contenus dans les tenseurs du modèle :


In [8]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


In [9]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


In [10]:
total_params_gpt2 = (
total_params - sum(p.numel()
for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
f"considering weight tying: {total_params_gpt2:,}"
)

Number of trainable parameters considering weight tying: 124,412,160


Pour finir, calculons l'empreinte mémoire des 163 millions de paramètres de notre objet `GPTModel` :


In [15]:
# Calculates the total size in bytes (assuming float32, 4 bytes per parameter)
total_size_bytes = total_params*4
total_size_mb = total_size_bytes/(1024*1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.83 MB


Ainsi, en supposant que chaque paramètre est un flottant 32 bits occupant 4 octets, la taille totale du modèle atteint 621,83 Mo, ce qui illustre l'espace de stockage déjà conséquent requis par un LLM pourtant relativement petit.


### Générer du texte


> Fonction permettant au modèle GPT de générer du texte


In [16]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=1)
        
    return idx

In [21]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


Nous plaçons ensuite le modèle en mode `.eval()`. Cela désactive les composantes aléatoires comme le dropout, réservé à l'entraînement, puis nous appliquons `generate_text_simple` sur le tenseur d'entrée encodé :


In [22]:
model.eval() #Disables dropout since we are not training the model
out = generate_text_simple(
    model=model,
    idx=encoded_tensor,
    max_new_tokens=6,
    context_size=GPT2_CONFIG_124M.context_length
)
print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10


Avec la méthode `.decode` du tokenizer, nous pouvons reconvertir les identifiants en texte :


In [23]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Featureiman Byeswickattribute argue


On constate que le modèle génère du charabia, bien loin d'un texte cohérent du type « Hello, I am a model ready to help. What happened? ». La raison est simple : le modèle n'a pas encore été entraîné.
